# Visualization

This tutorial will show some of the available interactive visualizations that are available in (or compatible with) SpikeInterface.

In [5]:
from pathlib import Path


import spikeinterface as si
import spikeinterface.preprocessing as spre
import spikeinterface.postprocessing as spost
import spikeinterface.curation as scur
import spikeinterface.widgets as sw
import spikeinterface.qualitymetrics

In [6]:
si.set_global_job_kwargs(n_jobs=-1, progress_bar=True)

Let's first load some data:

In [7]:
base_folder = Path('/home/samuel/DataSpikeSorting/gdr2024_tutorials/')

In [9]:
postprocessing_folder = base_folder / "postprocessing_data"

recording = si.load_extractor(postprocessing_folder / "recording")
sorting = si.load_extractor(postprocessing_folder / "sorting_mysterious")

In [11]:
recording

BinaryFolderRecording: 32 channels - 30.0kHz - 1 segments - 4,500,000 samples 
                       150.00s (2.50 minutes) - float32 dtype - 549.32 MiB

In [12]:
sorting

NumpyFolder: 74 units - 1 segments - 30.0kHz

## sortingview


[sortingview](https://github.com/magland/sortingview) is a Python package developed by Jeremy Magland that visualizes ephys data in the cloud.

It uses the [figurl](https://github.com/flatironinstitute/figurl/blob/main/README.md) technology to push the visualization data to the cloud and produce self-contained URL link that points will allow you to visualize and interact with the data from any computer connected to the internet.

Pretty cool, right??

To get started, you just need to run this command, click on the link, and login with your GitHub account:

In [15]:
!kachery-cloud-init

This client has already been registered.
Click the following link to configure the client:
https://kachery-gateway.figurl.org/client/6cd89860530568052bfbe3b89694bbdae8802105fa03871595358e9359f88f62

Client ID: 6cd89860530568052bfbe3b89694bbdae8802105fa03871595358e9359f88f62
Label: dell-sam
Owner: samuelgarcia

* Kachery-cloud is intended for collaborative sharing of data for scientific research. It should not be used for other purposes.


There are a few plot_* functions that support `sortingview` as backend, but the two most useful ones are the:

- `plot_traces()`
- `plot_sorting_summary()`

### plot_traces()

In [17]:
# let's do some preprocessing
recording = spre.depth_order(recording)
recording_hp = spre.highpass_filter(recording)
recording_cmr = spre.common_reference(recording_hp)


recording_layers = dict(
    raw=recording,
    highpass=recording_hp,
    cmr=recording_cmr
)

w = sw.plot_traces(
    recording_layers,
    mode="map",
    order_channel_by_depth=True,
    time_range=[0, 0.2], 
    figlabel="SpikeInterface tutorial: plot_traces",
    clim=(-50, 50),
    backend="sortingview"
)

https://figurl.org/f?v=npm://@fi-sci/figurl-sortingview@12/dist&d=sha1://ee7d63d5adfd3d982f063142ab607285eb452708&label=SpikeInterface%20tutorial%3A%20plot_traces


# plot_sorting_summary

For this plot, we need an analyzer and some extensions:

In [18]:
analyzer = si.create_sorting_analyzer(sorting, recording)

estimate_sparsity:   0%|          | 0/150 [00:00<?, ?it/s]

In [19]:
required_extensions = [
    "random_spikes",
    "waveforms",
    "templates",
    "noise_levels",
    "unit_locations",
    "template_similarity",
    "spike_amplitudes",
    "correlograms"
]
analyzer.compute(required_extensions)
analyzer

compute_waveforms:   0%|          | 0/150 [00:00<?, ?it/s]

noise_level:   0%|          | 0/20 [00:00<?, ?it/s]

Compute : spike_amplitudes:   0%|          | 0/150 [00:00<?, ?it/s]

SortingAnalyzer: 32 channels - 74 units - 1 segments - memory - sparse - has recording
Loaded 8 extensions: random_spikes, waveforms, templates, noise_levels, unit_locations, template_similarity, correlograms, spike_amplitudes

We can add any property to the units table:

In [20]:
num_spikes = sorting.count_num_spikes_per_unit()
unit_amplitudes = si.get_template_extremum_amplitude(analyzer)

analyzer.sorting.set_property("num_spikes", list(num_spikes.values()))
analyzer.sorting.set_property("amplitude", list(unit_amplitudes.values()))

In [21]:
w = sw.plot_sorting_summary(
    analyzer,
    unit_table_properties=["num_spikes", "amplitude"], 
    curation=True, 
    label_choices=["noise", "MUA", "SUA"],
    backend="sortingview"
)

https://figurl.org/f?v=npm://@fi-sci/figurl-sortingview@12/dist&d=sha1://a44d47eb90238bdf00adecd5ab48ba2d48a3c061


After curating, we can apply the curation either from the downloaded JSON file or with the URI:

In [25]:
curation_json = "sorting-curation.json"

sorting_curated_from_json = scur.apply_sortingview_curation(sorting, uri_or_json=curation_json)
sorting_curated_from_json

NumpySorting: 73 units - 1 segments - 30.0kHz

In [26]:
uri = "sha1://090dcb14f4a57781862be59a20ed5732b8341cdd"
sorting_curated_from_uri = scur.apply_sortingview_curation(sorting, uri_or_json=uri)
sorting_curated_from_uri

NumpySorting: 73 units - 1 segments - 30.0kHz

## GUIs

In [27]:
analyzer_saved = analyzer.save_as(
    folder=base_folder / "analyzer_for_visualization",
)

### plot_traces with `ephyviewer`

In [28]:
%gui qt
sw.plot_traces(recording, backend="ephyviewer")

/home/samuel/.venvs/py3.12/lib/python3.12/site-packages/h5py/__init__.py:36: UserWarning: h5py is running against HDF5 1.10.10 when it was built against 1.14.4, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


### plot_sorting_summary with `spikeinterface-gui`

In [29]:
analyzer_saved.compute("principal_components", n_components=3, mode="by_channel_global")
analyzer_saved.compute("quality_metrics", metric_names=["snr"])
analyzer_saved

Fitting PCA:   0%|          | 0/74 [00:00<?, ?it/s]

Projecting waveforms:   0%|          | 0/74 [00:00<?, ?it/s]

SortingAnalyzer: 32 channels - 74 units - 1 segments - memory - sparse - has recording
Loaded 10 extensions: random_spikes, waveforms, templates, noise_levels, unit_locations, template_similarity, correlograms, spike_amplitudes, principal_components, quality_metrics

In [30]:
%gui qt
sw.plot_sorting_summary(analyzer_saved, backend="spikeinterface_gui")